In [36]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

import matplotlib.pyplot as plt  # To visualize
import json
from collections import Counter, defaultdict

from sklearn.linear_model import LinearRegression
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', 200)

import warnings
warnings.filterwarnings('ignore')

from itertools import chain
import scipy.stats

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [92]:
df= pd.read_csv("../data/finals/colex_wn_pron_features.csv")

In [93]:
len(set(df["LANG_PRON"].tolist()))

142

In [95]:
df

,SENSE_LEMMA,LANG_PRON,PRON,COLEX,Conc.Mean,Conc.SD,V.Mean,V.SD,A.Mean,A.SD,D.Mean,D.SD,Aff.Mean,Aff.all.Mean,NORM_PRON,INIT_NORM_PRON,INIT_PRON,init_syl,init_son,init_cons,init_cont,init_delrel,init_lat,init_nas,init_strid,init_voi,init_sg,init_cg,init_ant,init_cor,init_distr,init_lab,init_hi,init_lo,init_back,init_round,init_velaric,init_tense,init_long,init_hitone,init_hireg,syl,son,cons,cont,delrel,lat,nas,strid,voi,sg,cg,ant,cor,distr,lab,hi,lo,back,round,velaric,tense,long,hitone,hireg,last_syl,last_son,last_cons,last_cont,last_delrel,last_lat,last_nas,last_strid,last_voi,last_sg,last_cg,last_ant,last_cor,last_distr,last_lab,last_hi,last_lo,last_back,last_round,last_velaric,last_tense,last_long,last_hitone,last_hireg,SEG_LEN,Family,vowels,vowel_ratio,PRON_TTR,PRON_complex,NORM_PRON_TTR,NORM_PRON_complex
0,a,eng,ə,alpha~settling,2.035,1.230,5.500,2.240,3.670,2.630,6.320,2.480,5.163333,0.490352,NaN,NaN,ə,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,-1,1,-1,-1,-1,-1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,-1,1,-1,-1,-1,-1,0,0,1,Indo-European,1,1.000000,1.000000,0.000000,NaN,NaN
1,a,hun,ɒː,amplitude~elevator,3.855,0.985,5.950,1.500,3.650,2.250,4.920,2.260,4.840000,0.654389,a,a,ɒ,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,1,1,1,-1,1,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,1,1,1,-1,1,1,0,0,2,Uralic,1,1.000000,1.000000,0.000000,1.000000,0.000000
2,a,hun,ɒː,amplitude~legislature,2.730,1.360,3.550,1.570,3.220,2.180,4.480,1.990,3.750000,0.481333,a,a,ɒ,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,1,1,1,-1,1,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,1,1,1,-1,1,1,0,0,2,Uralic,1,1.000000,1.000000,0.000000,1.000000,0.000000
3,a,hun,ɒː,amplitude~menopause,3.055,1.405,1.890,1.020,3.550,2.240,4.320,2.810,3.253333,0.486241,a,a,ɒ,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,1,1,1,-1,1,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,1,1,1,-1,1,1,0,0,2,Uralic,1,1.000000,1.000000,0.000000,1.000000,0.000000
4,a,hun,ɒː,amplitude~movie,3.755,1.185,7.240,1.510,4.390,2.630,5.680,2.160,5.770000,0.696056,a,a,ɒ,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,1,1,1,-1,1,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,1,1,1,-1,1,1,0,0,2,Uralic,1,1.000000,1.000000,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339687,𐌿𐍂𐍂𐌿𐌽𐍃,got,u r r u n s,dawn~departure,3.765,1.170,5.395,1.750,3.375,2.535,5.035,2.615,4.601667,0.632148,urruns,u,u,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,1,1,-1,1,1,-1,1,-1,0,0,2,5,4,5,0,0,1,0,5,0,0,4,4,0,2,2,0,2,2,0,2,0,0,0,-1,-1,1,1,-1,-1,-1,0,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,0,6,Indo-European,2,0.333333,0.666667,0.333333,0.666667,0.333333
339688,𐍃𐌺𐌴𐌹𐌼𐌰,got,s k iː m a,lantern~torch,4.880,0.320,4.935,1.700,4.165,2.195,5.550,2.160,4.883333,0.759296,skima,s,s,-1,-1,1,1,-1,-1,-1,0,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,0,2,3,3,3,0,0,1,0,3,0,0,2,1,0,1,2,1,1,0,0,2,1,0,0,1,1,-1,1,-1,-1,-1,0,1,-1,-1,0,-1,0,-1,-1,1,-1,-1,-1,1,-1,0,0,6,Indo-European,2,0.400000,1.000000,0.000000,1.000000,0.000000
339689,𐍅𐌰𐌹𐌽𐌰𐌷𐍃,got,w ɛː n a h s,miserable~unhappy,2.005,1.320,2.220,1.485,5.080,2.720,3.785,2.170,3.695000,0.405778,wenahs,w,w,-1,1,-1,1,-1,-1,-1,0,1,-1,-1,-1,-1,0,1,1,-1,1,1,-1,0,-1,0,0,2,5,3,5,0,0,1,0,4,0,0,2,2,0,1,1,1,1,1,0,1,1,0,0,-1,-1,1,1,-1,-1,-1,0,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,0,7,Indo-European,3,0.500000,1.000000,0.000000,1.000000,0.000000
339690,𐍅𐍉𐌸𐌴𐌹𐍃,got,w oː θ iː s,pleasant~sweet,2.775,1.020,7.505,1.445,3.525,2.720,6.410,1.965,5.81333

In [96]:
df["pron_lang"]=df["PRON"]+"~"+df["LANG_PRON"]

In [97]:
df["pron_lang"].drop_duplicates()

0                             ə~eng
1                            ɒː~hun
27                            a~ita
30                    aː i̯ ə n~nld
33                         aː l~nld
                    ...            
339687              u r r u n s~got
339688               s k iː m a~got
339689             w ɛː n a h s~got
339690              w oː θ iː s~got
339691    f u l l a f a h j a n~got
Name: pron_lang, Length: 84084, dtype: object

In [98]:
len(set(df["LANG_PRON"]))

142

In [87]:
df = pd.read_csv("../data/finals/colex_wn_pron_aff.csv")

In [88]:
len(df)

1980472

In [90]:
df["lemma_lang"]= df["SENSE_LEMMA"]+"~"+df["lang"]

In [91]:
df.drop_duplicates("lemma_lang")

,SENSE_LEMMA,LANG,SYNSET1,SYNSET2,C1,C2,diff,lang,C1.lemma,C1.V.Mean.Sum,C1.V.SD.Sum,C1.V.Rat.Sum,C1.A.Mean.Sum,C1.A.SD.Sum,C1.A.Rat.Sum,C1.D.Mean.Sum,C1.D.SD.Sum,C1.D.Rat.Sum,C2.lemma,C2.V.Mean.Sum,C2.V.SD.Sum,C2.V.Rat.Sum,C2.A.Mean.Sum,C2.A.SD.Sum,C2.A.Rat.Sum,C2.D.Mean.Sum,C2.D.SD.Sum,C2.D.Rat.Sum,LANG_PRON,LANG_NAME,LEN,lemma_lang,PRON
0,tropidoclonion_lineatum,BG,Tropidoclonion#n#1,lined_snake#n#1,tropidoclonion,lined_snake,True,bg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bul,Bulgarian,3.0,tropidoclonion_lineatum~bg,NaN
1,tropidoclonion_lineatum,RU,Tropidoclonion#n#1,lined_snake#n#1,tropidoclonion,lined_snake,True,ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rus,Russian,3.0,tropidoclonion_lineatum~ru,NaN
2,tropidoclonion_lineatum,UK,Tropidoclonion#n#1,lined_snake#n#1,tropidoclonion,lined_snake,True,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ukr,Ukrainian,3.0,tropidoclonion_lineatum~uk,NaN
3,tropidoclonion_lineatum,GA,Tropidoclonion#n#1,lined_snake#n#1,tropidoclonion,lined_snake,True,ga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gle,Irish,3.0,tropidoclonion_lineatum~ga,NaN
4,tropidoclonion_lineatum,RO,Tropidoclonion#n#1,lined_snake#n#1,tropidoclonion,lined_snake,True,ro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ron,Romanian; Moldavian; Moldovan,3.0,tropidoclonion_lineatum~ro,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980467,menuil,MS,jack#v#1,pry#v#1,jack,pry,True,ms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pry,3.57,1.86,21.0,5.3,2.58,20.0,4.85,2.25,20.0,msa,Malay (macrolanguage),3.0,menuil~ms,NaN
1980468,menuas,MS,jack#v#1,pry#v#1,jack,pry,True,ms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pry,3.57,1.86,21.0,5.3,2.58,20.0,4.85,2.25,20.0,msa,Malay (macrolanguage),3.0,menuas~ms,NaN
1980469,menuil,ID,jack#v#1,pry#v#1,jack,pry,True,id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pry,3.57,1.86,21.0,5.3,2.58,20.0,4.85,2.25,20.0,ind,Indonesian,3.0,menuil~id,NaN
1980470,menuas,ID,jack#v#1,pry#v#1,jack,pry,True,id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pry,3.57,1.86,21.0,5.3,2.58,20.0,4.85,2.25,20.0,ind,Indonesian,3.0,menuas~id,NaN


In [42]:
df["COLEX"]=df["C1"]+"~"+df["C2"]

In [44]:
def sort_colex(x):
    c1, c2 = x.split("~")
    sorted_l = sorted([c1,c2])
    x1,x2 = sorted_l
    return f"{x1}~{x2}"

In [46]:
df = df.dropna(subset=["COLEX"])

In [47]:
df["COLEX"] = df["COLEX"].apply(sort_colex)

In [52]:
len(df)

1980362

In [63]:
df

,SENSE_LEMMA,LANG,SYNSET1,SYNSET2,C1,C2,diff,lang,C1.lemma,C1.V.Mean.Sum,C1.V.SD.Sum,C1.V.Rat.Sum,C1.A.Mean.Sum,C1.A.SD.Sum,C1.A.Rat.Sum,C1.D.Mean.Sum,C1.D.SD.Sum,C1.D.Rat.Sum,C2.lemma,C2.V.Mean.Sum,C2.V.SD.Sum,C2.V.Rat.Sum,C2.A.Mean.Sum,C2.A.SD.Sum,C2.A.Rat.Sum,C2.D.Mean.Sum,C2.D.SD.Sum,C2.D.Rat.Sum,LANG_PRON,LANG_NAME,LEN,lemma_lang,PRON,COLEX
0,tropidoclonion_lineatum,BG,Tropidoclonion#n#1,lined_snake#n#1,tropidoclonion,lined_snake,True,bg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bul,Bulgarian,3.0,tropidoclonion_lineatum_bul,NaN,lined_snake~tropidoclonion
1,tropidoclonion_lineatum,RU,Tropidoclonion#n#1,lined_snake#n#1,tropidoclonion,lined_snake,True,ru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rus,Russian,3.0,tropidoclonion_lineatum_rus,NaN,lined_snake~tropidoclonion
2,tropidoclonion_lineatum,UK,Tropidoclonion#n#1,lined_snake#n#1,tropidoclonion,lined_snake,True,uk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ukr,Ukrainian,3.0,tropidoclonion_lineatum_ukr,NaN,lined_snake~tropidoclonion
3,tropidoclonion_lineatum,GA,Tropidoclonion#n#1,lined_snake#n#1,tropidoclonion,lined_snake,True,ga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gle,Irish,3.0,tropidoclonion_lineatum_gle,NaN,lined_snake~tropidoclonion
4,tropidoclonion_lineatum,RO,Tropidoclonion#n#1,lined_snake#n#1,tropidoclonion,lined_snake,True,ro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ron,Romanian; Moldavian; Moldovan,3.0,tropidoclonion_lineatum_ron,NaN,lined_snake~tropidoclonion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980467,menuil,MS,jack#v#1,pry#v#1,jack,pry,True,ms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pry,3.57,1.86,21.0,5.3,2.58,20.0,4.85,2.25,20.0,msa,Malay (macrolanguage),3.0,menuil_msa,NaN,jack~pry
1980468,menuas,MS,jack#v#1,pry#v#1,jack,pry,True,ms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pry,3.57,1.86,21.0,5.3,2.58,20.0,4.85,2.25,20.0,msa,Malay (macrolanguage),3.0,menuas_msa,NaN,jack~pry
1980469,menuil,ID,jack#v#1,pry#v#1,jack,pry,True,id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pry,3.57,1.86,21.0,5.3,2.58,20.0,4.85,2.25,20.0,ind,Indonesian,3.0,menuil_ind,NaN,jack~pry
1980470,menuas,ID,jack#v#1,pry#v#1,jack,pry,True,id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pry,3.57,1.86,21.0,5.3,2.58,20.0,4.85,2.25,20.0,ind,Indonesian,3.0,menuas_ind,NaN,jack~pry


In [54]:
df = df.drop_duplicates(subset=["SENSE_LEMMA", "LANG", "COLEX", "PRON"])

In [56]:
len(df)

1866558

In [55]:
len(df["COLEX"].value_counts())

676594

In [57]:
concepts = df["C1"].tolist()+df["C2"].tolist()

In [59]:
len(list(set(concepts)))

68249

In [60]:
SYNS = df["SYNSET1"].tolist()+df["SYNSET2"].tolist()

In [61]:
len(list(set(SYNS)))

72604

In [82]:
df_1 = df.dropna(subset=["C1", "C1.V.Mean.Sum"])
df_2 = df.dropna(subset=["C2", "C2.V.Mean.Sum"])

conc_dict = dict(zip(df_1["C1"], df_1["C1.V.Mean.Sum"]))
conc_dict2 = dict(zip(df_2["C2"], df_2["C2.V.Mean.Sum"]))
conc_dict.update(conc_dict2)

In [83]:
len(conc_dict)

10353

In [71]:
conc_dict["tropidoclonion"]

nan

In [73]:
l = [x for x, v in conc_dict.items() if v!="nan"]

In [68]:
len(l)

68249

In [77]:
df_wn = pd.read_csv("../data/aff+conc/colex_aff_conc_dist.csv")

In [35]:
df_wn[df_wn["COLEX"]=='santa_claus~dad']

,SENSE_LEMMA,LANG,SYNSET1,SYNSET2,C1,C2,Conc.M.1,Conc.M.2,lang,LANG_PRON,LANG_NAME,lemma_lang,PRON,Latitude,Longitude,COLEX,Conc.SD.1,Conc.SD.2,C1.lemma,C1.V.Mean.Sum,C1.V.SD.Sum,C1.V.Rat.Sum,C1.A.Mean.Sum,C1.A.SD.Sum,C1.A.Rat.Sum,C1.D.Mean.Sum,C1.D.SD.Sum,C1.D.Rat.Sum,C2.lemma,C2.V.Mean.Sum,C2.V.SD.Sum,C2.V.Rat.Sum,C2.A.Mean.Sum,C2.A.SD.Sum,C2.A.Rat.Sum,C2.D.Mean.Sum,C2.D.SD.Sum,C2.D.Rat.Sum,diff,Conc.Dist,V.Dist,D.Dist,A.Dist,Conc.Percent_Known.1,Conc.Percent_Known.2,D.SD.Dist,A.SD.Dist,V.SD.Dist,Conc.SD.Dist,Aff.Dist.Mean
150750,pare,CA,Santa_Claus#n#1,dad#n#1,santa_claus,dad,3.89,4.29,ca,spa,Castilian,pare_spa,p a ɾ e,40.4414,-1.11788,santa_claus~dad,1.29,1.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0.4,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,0.17,0.17
150863,babbo,IT,Santa_Claus#n#1,dad#n#1,santa_claus,dad,3.89,4.29,it,ita,Italian,babbo_ita,b a b b o,43.0464,12.64890,santa_claus~dad,1.29,1.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0.4,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,0.17,0.17
150866,papá,ES,Santa_Claus#n#1,dad#n#1,santa_claus,dad,3.89,4.29,es,spa,Spanish; Castilian,papá_spa,p a p a,40.4414,-1.11788,santa_claus~dad,1.29,1.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0.4,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,0.17,0.17


In [80]:
df_1 = df_wn.dropna(subset=["C1", "Conc.M.1"])
df_2 = df_wn.dropna(subset=["C2", "Conc.M.2"])

conc_dict = dict(zip(df_1["C1"], df_2["Conc.M.1"]))
conc_dict2 = dict(zip(df_2["C2"], df_2["Conc.M.2"]))
conc_dict.update(conc_dict2)

In [81]:
len(conc_dict)

19179

In [29]:
df_wn[df_wn["COLEX"].str.contains("dad")]["COLEX"].value_counts().to_dict()

{'dad~father': 89,
 'dad~pope': 19,
 'appliance~doodad': 13,
 'dad~sire': 10,
 'dad~forefather': 9,
 'scat~skedaddle': 7,
 'flee~skedaddle': 6,
 'catch~doodad': 6,
 'dad~fatherhood': 5,
 'dad~parent': 5,
 'doodad~matter': 4,
 'doodad~object': 4,
 'dad~pop_music': 3,
 'santa_claus~dad': 3,
 'commodity~doodad': 3,
 'escape~skedaddle': 3,
 'doodad~merchandise': 3,
 'dad~nipple': 3,
 'dad~package': 2,
 'dad~venerable': 2,
 'dad~previous': 2,
 'dad~experienced': 2,
 'dad~old': 2,
 'panama~dad': 2,
 'scram~skedaddle': 2,
 'doodad~material': 2,
 'cargo~doodad': 2,
 'crate~dad': 2,
 'dad~grandfather': 2,
 'dad~gruel': 2,
 'doodad~kind': 1,
 'dad~pauper': 1,
 'cinder~doodad': 1,
 'doodad~impurity': 1,
 'cheese~doodad': 1,
 'clientele~doodad': 1,
 'doodad~rubbish': 1,
 'doodad~waste': 1,
 'dad~popcorn': 1,
 'chap~dad': 1,
 'crazy~doodad': 1,
 'dad~sir': 1,
 'dad~pimp': 1,
 'doodad~plaything': 1,
 'doodad~happening': 1,
 'breadfruit~dad': 1,
 'chimpanzee~dad': 1,
 'bread~dad': 1,
 'antic~doodad':

In [2]:
df = pd.read_csv("../data/aff+conc/phone_aff_conc_classes.csv")

In [25]:
df[df["COLEX"]=='dad~pope']

,Unnamed: 0.1,Unnamed: 0,SENSE_LEMMA,LANG_PRON,PRON,COLEX,Conc.Mean,Conc.SD,V.Mean,V.SD,A.Mean,A.SD,D.Mean,D.SD,Aff.Mean,Aff.all.Mean,macroarae,Conc.CLASS,V.CLASS,A.CLASS,D.CLASS,Conc,V,A,D,Aff,Aff.all
187398,187398,187398,papa,ita,p a p a,dad~pope,4.5,0.915,6.16,2.115,4.13,2.375,5.18,2.455,5.156667,0.736481,Eurasia,2.0,2.0,1.0,1.0,3.0,5.0,3.0,4.0,3.0,-1.0
187404,187404,187404,papa,tgl,p a p a ʔ,dad~pope,4.5,0.915,6.16,2.115,4.13,2.375,5.18,2.455,5.156667,0.736481,Papunesia,2.0,2.0,1.0,1.0,3.0,5.0,3.0,4.0,3.0,-1.0
187406,187406,187406,papa,tur,p a p a,dad~pope,4.5,0.915,6.16,2.115,4.13,2.375,5.18,2.455,5.156667,0.736481,Eurasia,2.0,2.0,1.0,1.0,3.0,5.0,3.0,4.0,3.0,-1.0
289589,289589,289589,папа,rus,p a p ə,dad~pope,4.5,0.915,6.16,2.115,4.13,2.375,5.18,2.455,5.156667,0.736481,Eurasia,2.0,2.0,1.0,1.0,3.0,5.0,3.0,4.0,3.0,-1.0
292125,292125,292125,тато,ukr,t ɑ t ɔ,dad~pope,4.5,0.915,6.16,2.115,4.13,2.375,5.18,2.455,5.156667,0.736481,Eurasia,2.0,2.0,1.0,1.0,3.0,5.0,3.0,4.0,3.0,-1.0
297214,297214,297214,بابا,ara,b aː b aː,dad~pope,4.5,0.915,6.16,2.115,4.13,2.375,5.18,2.455,5.156667,0.736481,Eurasia,2.0,2.0,1.0,1.0,3.0,5.0,3.0,4.0,3.0,-1.0
309633,309633,309633,پاپ,fas,p ɔː p,dad~pope,4.5,0.915,6.16,2.115,4.13,2.375,5.18,2.455,5.156667,0.736481,Eurasia,2.0,2.0,1.0,1.0,3.0,5.0,3.0,4.0,3.0,-1.0


In [22]:
df[df["COLEX"].str.contains("dad")]["COLEX"].value_counts().to_dict()

{'dad~father': 67,
 'appliance~doodad': 13,
 'dad~forefather': 9,
 'dad~sire': 8,
 'dad~pope': 7,
 'catch~doodad': 6,
 'dad~parent': 5,
 'dad~fatherhood': 5,
 'doodad~matter': 4,
 'doodad~object': 4,
 'santa_claus~dad': 3,
 'dad~nipple': 3,
 'doodad~merchandise': 3,
 'commodity~doodad': 3,
 'dad~pop_music': 3,
 'cargo~doodad': 2,
 'scat~skedaddle': 2,
 'dad~venerable': 2,
 'dad~gruel': 2,
 'flee~skedaddle': 2,
 'panama~dad': 2,
 'dad~experienced': 2,
 'dad~previous': 2,
 'doodad~material': 2,
 'dad~old': 2,
 'dad~octopus': 1,
 'doodad~guy': 1,
 'doodad~ruse': 1,
 'antic~doodad': 1,
 'doodad~violin': 1,
 'doodad~kind': 1,
 'doodad~interest_rate': 1,
 'doodad~font': 1,
 'dad~popcorn': 1,
 'chap~doodad': 1,
 'dad~start': 1,
 'dad~grandfather': 1,
 'beget~dad': 1,
 'dad~tonic': 1,
 'dimwit~doodad': 1,
 'doodad~sheath': 1,
 'doodad~hull': 1,
 'doodad~scabbard': 1,
 'doodad~waste': 1,
 'doodad~rubbish': 1,
 'doodad~impurity': 1,
 'chap~dad': 1,
 'cinder~doodad': 1,
 'cheese~doodad': 1,
 'doo

In [8]:
df[df["COLEX"]=="fool~idiot"]["LANG_PRON"].value_counts()

LANG_PRON
nld    39
deu    21
fra    13
ita    12
rus    11
cmn     7
eng     7
fin     6
jpn     5
kor     5
spa     5
por     4
kat     4
glg     2
gle     2
msa     1
ind     1
tur     1
isl     1
ara     1
ben     1
tam     1
slv     1
swe     1
Name: count, dtype: int64

In [9]:
df_idiot = df[df["COLEX"]=="fool~idiot"]
df_idiot =df_idiot[df_idiot["LANG_PRON"]=="nld"]

In [15]:
df_idiot["SENSE_LEMMA"].values

array(['boerenlul', 'debiel', 'dommerik', 'domoor', 'drol', 'droplul',
       'eend', 'eendenkuiken', 'ei', 'eikel', 'ezel', 'ezelsveulen',
       'flapdrol', 'gehaktbal', 'gek', 'hals', 'hansworst', 'idioot',
       'imbeciel', 'kaffer', 'kalf', 'koe', 'kuiken', 'kwezel',
       'leeghoofd', 'oelewapper', 'oen', 'oetlul', 'oliebol', 'onbenul',
       'steenezel', 'stommeling', 'stommerd', 'sufferd', 'sufkut', 'uil',
       'uilenbal', 'uilskuiken', 'zakkenwasser'], dtype=object)